In [1]:
import pandas as pd
import numpy as np
import os

# Load Data sets

#### Load users

In [2]:
dtypes = {'id': 'str'}
user_file_path = os.path.join('..', 'data_in', 'User.csv')
users = pd.read_csv(user_file_path, dtype=dtypes)
users['dob_fix'] = pd.to_datetime(users.dob, errors='coerce')
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6729 entries, 0 to 6728
Data columns (total 5 columns):
id                       6729 non-null object
dob                      440 non-null object
cycle_length_initial     6729 non-null int64
period_length_initial    6729 non-null int64
dob_fix                  345 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 262.9+ KB


#### Check users
- still some issues

In [4]:
real_dates = users[users.dob_fix.notna()]
birth_years = real_dates.groupby(pd.Grouper(key='dob_fix', freq='y')).size().reset_index()
#lines_births = cm.line_chart(birth_years, x_col='dob_fix', y_col=0)
birth_years.plot.line(x='dob_fix', y=0)
%matplotlib inline 

#### Load Symptoms

In [ ]:
file_path = os.path.join('..', 'data_in', 'Symptom.csv')
symptoms = pd.read_csv(file_path)
symptoms.rename(columns={'id': 'symptom_id'})
symptoms['date_fix'] = pd.to_datetime(symptoms.date, errors='coerce')
symptoms.info()

#### User responses

In [ ]:
symptoms.groupby('user_id').size().describe()

#### Read Period Data

In [ ]:
file_path = os.path.join('..', 'data_in', 'Period.csv')
periods = pd.read_csv(file_path)
periods.rename(columns={'id': 'period_id'})
periods.columns = periods.columns.str.lower()
periods['start_date_fix'] = pd.to_datetime(periods.start_date, errors='coerce', dayfirst=True)
periods['end_date_fix'] = pd.to_datetime(periods.end_date, errors='coerce', dayfirst=True)
periods.info()

#### add period length

In [ ]:
periods['length'] = periods.end_date_fix - periods.start_date_fix
periods['length'].describe()

#### fix dates

In [ ]:
periods[['start_date_fix', 'end_date_fix']].describe()

#### Periods with dates over today
- why would there be data from 1958-61

In [ ]:
periods.loc[periods.start_date_fix > pd.to_datetime('today'), 'start_date'].str[-2:].value_counts()

#### A lot with period less than 0

In [ ]:
negatives_periods = periods[periods['length'] < pd.to_timedelta(0, unit='days')] 
print('Periods with negative times:', negatives_periods.shape[0])
negatives_periods.sample(5)


#### periods with long length

In [ ]:
max_days = 20
long_periods = periods[periods['length'] > pd.to_timedelta(max_days, unit='days')]
print('Count of periods over {} days: {}'.format(max_days, long_periods.shape[0]))

# Clean dates for periods
- drop anything under 0 days in length
- drop any period with a start or end date before toda

In [ ]:
print('Predrop:', periods.shape[0])
clean_periods = periods[periods.length >= pd.to_timedelta(0, unit='days')].copy()
print('after dropping negative period length:', clean_periods.shape[0])
clean_periods = clean_periods[clean_periods.start_date_fix < pd.to_datetime('today')]
print('after dropping start dates before today:', clean_periods.shape[0])
clean_periods = clean_periods[clean_periods.end_date_fix < pd.to_datetime('today')]
print('after dropping end dates before today:', clean_periods.shape[0])
max_days = 20
clean_periods = clean_periods[clean_periods['length'] < pd.to_timedelta(max_days, unit='days')]
print('after dropping periods over {} days: {}'.format(max_days, clean_periods.shape[0]))

In [ ]:
clean_periods.length.describe()